In [ ]:
# ! pip install langchain_openai
# ! pip install load_dotenv
# ! pip install tiktoken
# ! pip install langchain_huggingface
# ! pip install openpyxl
# ! pip install chromadb
# ! pip install rank_bm25

In [1]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEndpoint
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
import chromadb

from openpyxl import load_workbook
from typing import Dict, List, Optional
from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader

c:\Users\SUNJIN\anaconda3\envs\intern\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
api_key = os.environ.get('Openai_key')

사용할 데이터 불러오기

In [8]:
# class XLSXLoader(BaseLoader):
#     """Loads an XLSX file into a list of documents from specified sheets.

#     Each document represents one row of the XLSX file. Every row is converted into a
#     key/value pair and outputted to a new line in the document's page_content, including
#     the sheet name.

#     The source for each document loaded from xlsx is set to the value of the
#     'file_path' argument for all documents by default.
#     You can override this by setting the 'source_column' argument to the
#     name of a column in the XLSX file.
#     The source of each document will then be set to the value of the column
#     with the name specified in 'source_column'.

#     Output Example:
#         .. code-block:: txt

#             sheet: SheetName
#             column1: value1
#             column2: None
#             column3: value3
#     """

#     docs = []

#     def __init__(
#             self,
#             file_path: str,
#             source_column: Optional[str] = None,
#             encoding: Optional[str] = None,
#             exclude_columns: Optional[List[str]] = None,  # 열을 제외할 목록
#             target_sheets: Optional[List[str]] = None  # 불러올 시트 목록
#     ):
#         self.file_path = file_path
#         self.source_column = source_column
#         self.encoding = encoding
#         self.exclude_columns = exclude_columns if exclude_columns else []
#         self.target_sheets = target_sheets if target_sheets else []

#     def load(self) -> List[Document]:
#         docs = []

#         wb = load_workbook(filename=self.file_path, read_only=True, data_only=True)

#         # 특정 시트를 처리
#         sheets_to_process = self.target_sheets if self.target_sheets else wb.sheetnames[1:]


#         for sheet_name in sheets_to_process:
#             if sheet_name in wb.sheetnames:
#                 ws = wb[sheet_name]
#                 headers = [cell.value for cell in ws[1]]
#                 headers = [header if header is not None else '' for header in headers]  # None을 빈 문자열로 변환

#                 # 이전 행의 값을 저장하기 위한 리스트
#                 previous_row = [None] * len(headers)

#                 for i, row in enumerate(ws.iter_rows(min_row=2)):
#                     row_values = [cell.value for cell in row]

#                     # None 값을 이전 행의 값으로 대체, 제외할 열을 처리
#                     for j, (value, header) in enumerate(zip(row_values, headers)):
#                         if value is None and header not in self.exclude_columns:
#                             row_values[j] = previous_row[j]

#                         # '단종 여부' 열 처리
#                         if header == '단종 여부' and value is None:
#                             row_values[j] = '해당없음'

#                         # '구분' 열 처리
#                         if header == '구분':
#                             if value is None:
#                                 row_values[j] = previous_row[j]
#                             if 'ODM' in (row_values[j] or ''):
#                                 row_values[j] = 'ODM'
#                             else:
#                                 row_values[j] = 'OEM'

#                         # '런칭여부' 열 처리
#                         if header == '런칭여부':
#                             if value is None:
#                                 row_values[j] = previous_row[j]
#                             # else 문을 제거하여 중복되는 처리를 피함

#                     row_dict = dict(zip(headers, row_values))


#                     # # 시트 이름과 None 값을 처리하여 'None' 문자열로 변환
#                     # content_lines = [f"sheet: {sheet_name}"]  # 시트 이름 추가
#                     # content_lines.extend(
#                     #     f"{k.strip()}: {str(v) if v is not None else 'None'}"
#                     #     for k, v in row_dict.items()
#                     #     if k.strip() != ''  # 빈 헤더 무시
#                     # )
#                     # content = "\n".join(content_lines)

#                     if self.source_column is not None:
#                         source = row_dict.get(self.source_column, 'None')  # source_column이 없는 경우 'None' 사용
#                     else:
#                         source = self.file_path

#                     lines = f"""제품의 이름은 {row_dict['제품명']}입니다. 해당 제품의 채널, 고객사는 {row_dict['채널, 고객사']}이며,단종 여부는 {row_dict['단종 여부']}입니다. 제품의 런칭여부는 {row_dict['런칭여부']}입니다. 제품은 {row_dict['구분']}으로 구분되어있으며, 제품의 사용원료는 {row_dict['사용원료']}입니다. 제품의 컨셉은 {row_dict['컨셉']}입니다."""


#                     metadata = {"source": source, "sheet": sheet_name}
#                     row_dict = Document(page_content=lines, metadata=metadata)
#                     docs.append(row_dict)

#                     # 현재 행의 값을 이전 행으로 저장
#                     previous_row = row_values

#         return docs


In [9]:
# class XLSXLoader(BaseLoader):
#     """Loads an XLSX file into a list of documents from specified sheets.

#     Each document represents one row of the XLSX file. Every row is converted into a
#     key/value pair and outputted to a new line in the document's page_content, including
#     the sheet name.

#     The source for each document loaded from xlsx is set to the value of the
#     'file_path' argument for all documents by default.
#     You can override this by setting the 'source_column' argument to the
#     name of a column in the XLSX file.
#     The source of each document will then be set to the value of the column
#     with the name specified in 'source_column'.

#     Output Example:
#         .. code-block:: txt

#             sheet: SheetName
#             column1: value1
#             column2: None
#             column3: value3
#     """

#     docs = []

#     def __init__(
#             self,
#             file_path: str,
#             source_column: Optional[str] = None,
#             encoding: Optional[str] = None,
#             exclude_columns: Optional[List[str]] = None,  # 열을 제외할 목록
#             target_sheets: Optional[List[str]] = None  # 불러올 시트 목록
#     ):
#         self.file_path = file_path
#         self.source_column = source_column
#         self.encoding = encoding
#         self.exclude_columns = exclude_columns if exclude_columns else []
#         self.target_sheets = target_sheets if target_sheets else []

#     def load(self) -> List[Document]:
#         docs = []

#         wb = load_workbook(filename=self.file_path, read_only=True, data_only=True)

#         # 특정 시트를 처리
#         sheets_to_process = self.target_sheets if self.target_sheets else wb.sheetnames[1:]


#         for sheet_name in sheets_to_process:
#             if sheet_name in wb.sheetnames:
#                 ws = wb[sheet_name]
#                 headers = [cell.value for cell in ws[1]]
#                 headers = [header if header is not None else '' for header in headers]  # None을 빈 문자열로 변환

#                 # 이전 행의 값을 저장하기 위한 리스트
#                 previous_row = [None] * len(headers)

#                 for i, row in enumerate(ws.iter_rows(min_row=2)):
#                     row_values = [cell.value for cell in row]

#                     # None 값을 이전 행의 값으로 대체, 제외할 열을 처리
#                     for j, (value, header) in enumerate(zip(row_values, headers)):
#                         if value is None and header not in self.exclude_columns:
#                             row_values[j] = previous_row[j]

#                         # '단종 여부' 열 처리
#                         if header == '단종 여부' and value is None:
#                             row_values[j] = '미단종'

#                         # '구분' 열 처리
#                         if header == '구분':
#                             if value is None:
#                                 row_values[j] = previous_row[j]
#                             if 'ODM' in (row_values[j] or ''):
#                                 row_values[j] = 'ODM'
#                             else:
#                                 row_values[j] = 'OEM'

#                         # '런칭여부' 열 처리
#                         if header == '런칭여부':
#                             if value is None:
#                                 row_values[j] = previous_row[j]
#                             # else 문을 제거하여 중복되는 처리를 피함

#                     row_dict = dict(zip(headers, row_values))


#                     # # 시트 이름과 None 값을 처리하여 'None' 문자열로 변환
#                     # content_lines = [f"sheet: {sheet_name}"]  # 시트 이름 추가
#                     # content_lines.extend(
#                     #     f"{k.strip()}: {str(v) if v is not None else 'None'}"
#                     #     for k, v in row_dict.items()
#                     #     if k.strip() != ''  # 빈 헤더 무시
#                     # )
#                     # content = "\n".join(content_lines)

#                     # if self.source_column is not None:
#                     #     source = row_dict.get(self.source_column, 'None')  # source_column이 없는 경우 'None' 사용
#                     # else:
#                     #     source = self.file_path

#                     # lines = f"""제품의 이름은 {row_dict['제품명']}입니다. 해당 제품의 채널, 고객사는 {row_dict['채널, 고객사']}이며,단종 여부는 {row_dict['단종 여부']}입니다. 제품의 런칭여부는 {row_dict['런칭여부']}입니다. 제품은 {row_dict['구분']}으로 구분되어있으며, 제품의 사용원료는 {row_dict['사용원료']}입니다. 제품의 컨셉은 {row_dict['컨셉']}입니다."""


#                     # metadata = {"source": source, "sheet": sheet_name}
#                     # row_dict = Document(page_content=lines, metadata=metadata)
#                     docs.append(row_dict)

#                     # 현재 행의 값을 이전 행으로 저장
#                     previous_row = row_values

#         return docs


In [3]:
class XLSXLoader(BaseLoader):
    """Loads an XLSX file into a list of documents from specified sheets.

    Each document represents one row of the XLSX file. Every row is converted into a
    key/value pair and outputted to a new line in the document's page_content, including
    the sheet name.

    The source for each document loaded from xlsx is set to the value of the
    'file_path' argument for all documents by default.
    You can override this by setting the 'source_column' argument to the
    name of a column in the XLSX file.
    The source of each document will then be set to the value of the column
    with the name specified in 'source_column'.

    Output Example:
        .. code-block:: txt

            sheet: SheetName
            column1: value1
            column2: None
            column3: value3
    """

    docs = []

    def __init__(
            self,
            file_path: str,
            source_column: Optional[str] = None,
            encoding: Optional[str] = None,
            exclude_columns: Optional[List[str]] = None,  # 열을 제외할 목록
            target_sheets: Optional[List[str]] = None  # 불러올 시트 목록
    ):
        self.file_path = file_path
        self.source_column = source_column
        self.encoding = encoding
        self.exclude_columns = exclude_columns if exclude_columns else []
        self.target_sheets = target_sheets if target_sheets else []

    def load(self) -> List[Document]:
        docs = []

        wb = load_workbook(filename=self.file_path, read_only=True, data_only=True)

        # 특정 시트를 처리
        sheets_to_process = self.target_sheets if self.target_sheets else wb.sheetnames[1:]


        for sheet_name in sheets_to_process:
            if sheet_name in wb.sheetnames:
                ws = wb[sheet_name]
                headers = [cell.value for cell in ws[1]]
                headers = [header if header is not None else '' for header in headers]  # None을 빈 문자열로 변환

                # 이전 행의 값을 저장하기 위한 리스트
                previous_row = [None] * len(headers)

                for i, row in enumerate(ws.iter_rows(min_row=2)):
                    row_values = [cell.value for cell in row]

                    # None 값을 이전 행의 값으로 대체, 제외할 열을 처리
                    for j, (value, header) in enumerate(zip(row_values, headers)):
                        if value is None and header not in self.exclude_columns:
                            row_values[j] = previous_row[j]

                        # '단종 여부' 열 처리
                        if header == '단종 여부' and value is None:
                            row_values[j] = '해당없음'

                        # '구분' 열 처리
                        if header == '구분':
                            if value is None:
                                row_values[j] = previous_row[j]
                            if 'ODM' in (row_values[j] or ''):
                                row_values[j] = 'ODM'
                            else:
                                row_values[j] = 'OEM'

                        # '런칭여부' 열 처리
                        if header == '런칭여부':
                            if value is None:
                                row_values[j] = previous_row[j]
                            # else 문을 제거하여 중복되는 처리를 피함

                    row_dict = dict(zip(headers, row_values))


                    # 시트 이름과 None 값을 처리하여 'None' 문자열로 변환
                    content_lines = [f"sheet: {sheet_name}"]  # 시트 이름 추가
                    content_lines.extend(
                        f"{k.strip()}: {str(v) if v is not None else 'None'}"
                        for k, v in row_dict.items()
                        if k.strip() != ''  # 빈 헤더 무시
                    )
                    content = "\n".join(content_lines)

                    if self.source_column is not None:
                        source = row_dict.get(self.source_column, 'None')  # source_column이 없는 경우 'None' 사용
                    else:
                        source = self.file_path

                    # lines = f"""제품의 이름은 {row_dict['제품명']}입니다. 해당 제품의 채널, 고객사는 {row_dict['채널, 고객사']}이며,단종 여부는 {row_dict['단종 여부']}입니다. 제품의 런칭여부는 {row_dict['런칭여부']}입니다. 제품은 {row_dict['구분']}으로 구분되어있으며, 제품의 사용원료는 {row_dict['사용원료']}입니다. 제품의 컨셉은 {row_dict['컨셉']}입니다."""


                    metadata = {"source": source, "sheet": sheet_name}
                    row_dict = Document(page_content=content, metadata=metadata)
                    docs.append(row_dict)

                    # 현재 행의 값을 이전 행으로 저장
                    previous_row = row_values

        return docs

In [20]:
# 파일 불러오기
loader = XLSXLoader(
    file_path="C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/data/년도별 신제품 리스트_냉장-240425.xlsx",
    source_column="제품명",
    exclude_columns=["단종 여부"],  # 제외할 열 지정
    # target_sheets=["2023"]  # 불러올 시트 지정
)

docs = loader.load()

In [21]:
docs

[Document(metadata={'source': '멜팅피스 핫도그용소시지(오리지널)', 'sheet': '2023'}, page_content='sheet: 2023\n구분: OEM\n채널, 고객사: 외식자재\n제품명: 멜팅피스 핫도그용소시지(오리지널)\n단종 여부: 해당없음\n런칭여부: 런칭\n사용원료: 후지, 지방\n컨셉: 고기씹는 거친 식감과 육즙이 풍부한 핫도그용 소시지(멜팅피스)'),
 Document(metadata={'source': '멜팅피스 핫도그용소시지(치즈)', 'sheet': '2023'}, page_content='sheet: 2023\n구분: OEM\n채널, 고객사: 외식자재\n제품명: 멜팅피스 핫도그용소시지(치즈)\n단종 여부: 해당없음\n런칭여부: 런칭\n사용원료: 후지, 지방\n컨셉: 육 식감이 살아있고 다즙한 콘도그용 치즈 소시지'),
 Document(metadata={'source': '멜팅피스 핫도그용소시지(치즈&할라피뇨)', 'sheet': '2023'}, page_content='sheet: 2023\n구분: OEM\n채널, 고객사: 외식자재\n제품명: 멜팅피스 핫도그용소시지(치즈&할라피뇨)\n단종 여부: 해당없음\n런칭여부: 런칭\n사용원료: 후지, 지방\n컨셉: 육 식감이 살아있고 다즙한 콘도그용 치즈&할라피뇨 소시지'),
 Document(metadata={'source': '초당옥수수핫도그용소시지', 'sheet': '2023'}, page_content='sheet: 2023\n구분: OEM\n채널, 고객사: 학교급식\n제품명: 초당옥수수핫도그용소시지\n단종 여부: 해당없음\n런칭여부: 런칭\n사용원료: 후지, 지방\n컨셉: 육즙이 풍부한 학교급식용(아질산/MSG 제외) 미니 소시지(#21콜라겐 케이싱, 20g) '),
 Document(metadata={'source': '트리플슬라이스소시지', 'sheet': '2023'}, page_content='sheet: 2023\n구분: OEM\n채널, 고객사

In [13]:
# documents = []

# for doc in docs:
#     line = f"""{doc['제품명']}입니다. 해당 제품의 채널, 고객사는 {doc['채널, 고객사']}이며,단종 여부는 {doc['단종 여부']}입니다. 제품의 런칭여부는 {doc['런칭여부']}입니다. 제품은 {doc['구분']}으로 구분되어있으며, 제품의 사용원료는 {doc['사용원료']}입니다. 제품의 컨셉은 {doc['컨셉']}입니다."""
#     documents.append(line)

In [14]:
# documents

In [15]:
# # 토크나이징
# tokenizer = tiktoken.get_encoding("cl100k_base")

# def tiktoken_len(text):
#     tokens = tokenizer.encode(text)
#     return len(tokens)

In [16]:
# # chunking
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10, length_function=tiktoken_len)
# texts = text_splitter.split_documents(docs)

In [17]:
# # 벡터화에 사용할 모델(huggingFaceEmbeddings)
# model_name ="jhgan/ko-sbert-nli"
# # model_kwargs = {'device': 'cuda'} # gpu 사용
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# hf = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

In [23]:
# # # openaiEmbeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small", api_key=api_key)

# db = Chroma.from_documents(
#     docs, 
#     embeddings_model,
#     collection_name = 'history',
#     persist_directory = './db/chromadb_all_new',
#     collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
# )

# db

모델

In [47]:
# DB에 넣고 리트리버 구성
# docsearch = Chroma.from_documents(docs, hf) # chunking 을 안하면서 texts -> documents로
# retriever = db.as_retriever(    search_type='mmr',
#     search_kwargs={'k': 10, 'lambda_mult': 0.15})

In [8]:
db = Chroma(persist_directory="C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/db/chromadb_all_new",
            embedding_function = embeddings_model,
            collection_name = 'history',
            collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default)
)            

In [26]:
retriever = db.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.3,'k': 20}
)

In [27]:
d = retriever.get_relevant_documents("후지를 사용한 제품")
print(f"문서의 개수: {len(d)}")
d

문서의 개수: 20


[Document(metadata={'sheet': '2014', 'source': '흑후추소시지'}, page_content='sheet: 2014\n구분: OEM\n채널, 고객사: DLG\n제품명: 흑후추소시지\n단종 여부: 해당없음\n런칭여부: 뱅킹화\n사용원료: 후지, 지방\n컨셉: DLG 출품 목적 TG + 혼합믹싱 공정적용 후추가들어간소시지 extension'),
 Document(metadata={'sheet': '2023', 'source': '돈후지 불고기 2종'}, page_content='sheet: 2023\n구분: OEM\n채널, 고객사: 후레시스\n제품명: 돈후지 불고기 2종\n단종 여부: 해당없음\n런칭여부: 뱅킹화\n사용원료: 후지\n컨셉: 돈후지를 활용한 2가지 종류의 도시락용 비가열 불고기 제품 (간장/고추장)'),
 Document(metadata={'sheet': '2010', 'source': '후추가 들어간샐러드용소시지'}, page_content='sheet: 2010\n구분: OEM\n채널, 고객사: 신유통(홈쇼핑)\n제품명: 후추가 들어간샐러드용소시지\n단종 여부: 단종\n런칭여부: 런칭 \n사용원료: 후지, 지방, 돈피\n컨셉: 마일드한 입자형 훈연소시지'),
 Document(metadata={'sheet': '2016', 'source': '실속형후랑크'}, page_content='sheet: 2016\n구분: ODM\n채널, 고객사: 신세계푸드\n제품명: 실속형후랑크\n단종 여부: 해당없음\n런칭여부: 뱅킹화\n사용원료: 후지, 지방, MDCM\n컨셉: 기제품 (찌개용후랑크) 규격변경 extension 제품 '),
 Document(metadata={'sheet': '2016', 'source': '올가후랑크'}, page_content='sheet: 2016\n구분: ODM\n채널, 고객사: 풀무원\n제품명: 올가후랑크\n단종 여부: 해당없음\n런칭여부: 뱅킹화\n사용원료: 후지, 지방\n컨셉: 기제품 색

In [ ]:
# from langchain.retrievers import BM25Retriever

# BM25_retriever = BM25Retriever.from_texts(docs, k = 5)
# Chroma_retriever = db.as_retriever(
#     search_type='similarity_score_threshold',
#     search_kwargs={'score_threshold': 0.4}
# )

In [ ]:
# from langchain.retrievers import EnsembleRetriever

In [ ]:
# ensemble_retriever = EnsembleRetriever(
#                     retrievers = [BM25_retriever,Chroma_retriever]
#                     , weight = {0.5,0.5})

In [ ]:
# docs = ensemble_retriever.invoke("닭으로 만들어진 제품은?")
# docs

In [231]:
# llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
#     api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
#     model="xionic-ko-llama-3-70b",
#     temperature=0.75)

In [28]:
llm = ChatOpenAI(model_name="gpt-4o-mini",
                    streaming=True, callbacks=[StreamingStdOutCallbackHandler()],
                    temperature = 0.75,
                    max_tokens = 500,
                    api_key=api_key)

In [29]:
contextualize_q_system_prompt = (
      "당신은 회사 제품을 잘 알고 있는 마케딩 디렉터입니다."
      "답변은 사용자가 질문한 언어와 같은 언어를 사용하세요."
      "다음 질문에 대해 주어진 문맥을 사용하여 상세하고 정확한 답변을 제공해주세요. "
      "관련 문맥 부분을 참조하여 답변을 작성하세요.\n\n"
      "{context}"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        "{context}",
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [33]:
system_prompt = (
    """
    You MUST Answer in Korean.
    당신은 회사 제품을 잘 알고 있는 신제품 개발 팀장입니다.
    새로운 제품을 출시하기 위해 과거 신제품에 대한 정보를 모두 알고 있으며 팀원들이 특정 제품에 대해 물어볼 때 모두 대답할 수 있어야합니다.
    대답에 전체 제품이 아닌 몇가지를 보여준다는걸 꼭 명시해야 합니다.

    어떤 경우에도 {context} 에 있는 제품을 참고하여 답을 출력해야합니다.
    검색 결과를 낼 수 없다면 {context}에 있는 제품 중 첫번째 제품을 출력해야하고,
    다른 검색을 유도하는 "추가로 정보를 알고 싶으시다면 ~에 대해 검색해주세요"와 같은 말은 하지 말아야합니다.
    대답 후 엑셀 시트도 함께 출력해야하며 엑셀 시트는 {context} 에 있는 문서를 포함해야 합니다.
    아래 예시처럼 제품을 소개할 때 관련이 있는 제품은 모두 소개해야합니다.
    답변의 마지막에는 엑셀 형식으로 표를 만들어서 {context}에 있는 모든 문서를 포함해야 합니다.
    
    예를 들어, "학교에 납품하는 소시지 제품이 뭐가 있어?" 라고 물어본다면, 아래와 같이 답장하여야 합니다.

    납품처가 급식용으로 되어 있는 소시지 제품 몇가지를 보여드리겠습니다.

    첫번째 제품은 2023년도에 출시된 "초당옥수수핫도그용소시지" 입니다.
    해당 제품은 육즙이 풍부한 학교 급식용 미니 소시지로 아질산과 MSG를 제외하여 아이들도 걱정없이 먹을 수 있는 제품입니다.
    2023년도에 출시되어 아직까지 단종된 바 없으며 후지, 지방을 사용해 만들었습니다.
    
    두번째 제품은 2023년도에 출시된 "트리플슬라이스소시지" 입니다.
    해당 제품은 과거 단종된 제품인 "모듬슬라이스소시지"를 리뉴얼한 제품으로 갈비맛슬라이스소시지, 할라피뇨슬라이스소시지 신규 개발에 적용할 예정입니다.
    2023년도에 출시되어 아직까지 단종된 바 없으며 후지, 지방을 사용해 만들었습니다.

    이 밖에도 급식용 소시지 제품이 더 있을 수 있으며,
    더 다양한 제품의 정보를 알고 싶으시다면 소시지 제품에 대해 검색해주세요.

    | 구분  | 채널, 고객사   | 제품명                     | 단종 여부 | 런칭여부   | 사용원료           | 컨셉                                             |
    |-------|---------------|---------------------------|-----------|-----------|-------------------|--------------------------------------------------|
  
    표를 엑셀에 그대로 복사하시면 엑셀 형식으로 적용됩니다.

    예를 들어, "닭가슴살 제품이 뭐가 있어?" 라고 물어본다면, 아래와 같이 답장하여야 합니다.

    닭가슴살 제품 몇가지를 보여드리겠습니다.

    첫번째 제품은 2023년도에 출시된 "선진 닭가슴살슬라이스햄"입니다.
    해당 제품은 군납용 트레이 포장된 닭가슴살 슬라이스햄으로 육계가슴살을 사용하여 만들어졌습니다.
    2023년도에 출시되어 아직까지 단종된 바 없으며 군납 채널을 통해 판매되고 있습니다. 
    
    두번째 제품은 2023년도에 출시된 "레몬파슬리 닭가슴살소시지"입니다. 
    해당 제품은 일본 롱셀러 제품 레몬향이 나는 닭가슴살활용 레몬파슬리 소시지로 종계 가슴살을 사용하여 만들어졌습니다. 
    2023년도에 출시되어 아직까지 단종된 바 없으며 후레시스 채널을 통해 판매되고 있습니다. 
    
    세번째 제품은 2023년도에 출시된 "선진 허브갈릭닭가슴살소시지"입니다. 
    해당 제품은 계육 함량 80% 이상의 퍽퍽하지 않은 부드럽고 탄력있는 닭가슴살 소시지로 종계 가슴살을 사용하여 만들어졌습니다. 
    2023년도에 출시되어 아직까지 단종된 바 없으며 제이브로 채널을 통해 판매되고 있습니다. 
    
    이 밖에 더 많은 닭가슴살 제품이 있으며,
    더 다양한 제품 정보를 알고 싶으시다면 사용원료 종계 제품에 대해 검색해주세요.

    | 구분  | 채널, 고객사   | 제품명                     | 단종 여부 | 런칭여부   | 사용원료           | 컨셉                                             |
    |-------|---------------|---------------------------|-----------|-----------|-------------------|--------------------------------------------------|
   
    표를 엑셀에 그대로 복사하시면 엑셀 형식으로 적용됩니다.


    검색 결과를 낼 수 없다면 {context}에 있는 제품 중 첫번째 제품을 출력해야 합니다.

    예를 들어, 파스타와 관련된 {context} 를 찾을 수 없지만 "파스타 관련 제품이 뭐가 있어?" 라고 물어본다면, 아래와 같이 답장하여야 합니다.

    회사 제품 중 파스타는 없지만 파스타와 관련 있는 제품들을 보여드리겠습니다.

    첫번째 제품은 2023년도에 출시된 "라구소스토핑" 입니다.
    해당 제품은 볼로네제 풍미의 소스가 혼합된 냉동 피자용 돈육 토핑으로 후지를 사용하여 만들어졌습니다.
    2023년도에 출시되어 아직까지 단종된 바 없으며 풀무원에 유통되고 있습니다.

    파스타 소스로 많이 이용하는 라구 소스의 일종인 볼로네제를 활용했다는 점에서 파스타와 관련된 제품으로 검색되었습니다.

    두번째 제품은 2023년도에 출시된 "큐브찹스테이크" 입니다.
    해당 제품은 외주업체(에프와이지)가 생산하는 후지를 다이스하여 소스를 버무린 토핑용 제품으로 후지를 사용하여 만들어졌습니다.
    2023년도에 출시되어 현재는 단종이 되었으며 군납용으로 제작되었었습니다.

    파스타와 함께 양식으로 분류된다는 점에서 파스타와 관련된 제품으로 검색되었습니다.

    | 구분  | 채널, 고객사   | 제품명                     | 단종 여부 | 런칭여부   | 사용원료           | 컨셉                                             |
    |-------|---------------|---------------------------|-----------|-----------|-------------------|--------------------------------------------------|
   
    표를 엑셀에 그대로 복사하시면 엑셀 형식으로 적용됩니다.


    예를 들어, 떡볶이와 관련된 {context} 를 찾을 수 없지만 "떡볶이 관련 제품이 뭐가 있어?" 라고 물어본다면, 아래와 같이 답장하여야 합니다.

    현재는 떡볶이 제품이 없지만, 매콤한 소스에 버무려진 제품들이 있습니다. 
    
    첫번째 제품은 2023년도에 출시된 "매콤돼지구이" 입니다. 
    해당 제품은 후지를 사용하여 만들어진 매콤한 소스에 버무려진 다이스된 돼지구이 컨셉의 도시락 반찬입니다. 
    2023년도에 출시되어 아직까지 단종된 바 없으며 이마트 24 제안 최근 스펙 만족 피드백이 있습니다. 
    
    두번째 제품은 2023년도에 출시된 "유어스)바베큐폭립 -매콤한맛-" 입니다. 
    해당 제품은 로인립을 사용하여 만들어진 편의점 안주/간식용 매운맛 소스를 적용한 로인립입니다. 
    2023년도에 출시되어 시즌 한정 제품으로 조기 단종이 되었습니다. 
    
    이 밖에 더 많은 매콤한 소스 제품이 있으며, 추가로 떡볶이 관련 제품 정보를 알고 싶으시다면 소스 제품에 대해 검색해주세요.

    관련 {context}가 아예 나오지 않으면 아래와 같이 답장하여야 합니다.
    예를 들어, "발렌타인데이를 맞이하여 출시한 제품을 알려줘" 라고 물어본다면, 아래와 같이 답장하여야 합니다.
    현재는 찾는 조건에 해당하는 제품이 없습니다. 다른 제품을 검색해주세요.

    이용자가 검색 용도로 LLM을 사용할 수도 있지만 신메뉴에 대한 아이디어를 얻기 위해 사용하는 경우도 존재합니다.
    제품 검색을 위한 사용이 아니라고 판단 될 경우 대화를 나누고 관련 지식을 제공하는데 초점을 맞춰야합니다.
    회사가 육가공품을 제조하는 회사임을 고려하여 답장하여야합니다.

    예를 들어, "발렌타인데이를 맞이해서 신메뉴를 만들고 싶은데 추천해줘" 라고 물어본다면, 
    발렌타인데이는 발렌티노의 축일에서 유래한 기념일로 사랑하는 사람에게 초콜릿을 나눠주는 풍습이 있습니다.
    따라서, 초콜릿과 같은 달달한 간식 메뉴를 추천합니다.
    제품명은 "러블리치즈햄토스트"로 식빵안에 치즈와 햄 그리고 딸기잼을 넣은 토스트입니다.
    채널, 고객사는 편의점 또는 학교로 발렌타인데이를 맞이하여 납품하면 좋은 반응을 얻을 수 있을 것입니다. 

    예를 들어, "다이어트용 제품 어떤게 있어?" 라고 물어본다면,
    다이어트용으로는 닭가슴살이나 닭소세지 같은 닭을 사용한 제품이 인기가 많습니다.
    닭은 저지방 고단백 식품으로 다이어트에 적합하여 냉동 닭가슴살이나 조리된 제품을 쉽게 찾을 수 있으며,
    저지방 소고기의 경우 지방이 적은 부위인 안심이나 등심을 활용할 경우 다이어트에 도움이 될 수 있습니다.
    

    {context}
    
    이건 사용자의 질문입니다. {input}
    """
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        "{context}",
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [43]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [41]:
conversational_rag_chain.invoke(
    {"input": "베이컨 제품 있어?"},
    config={"configurable": {"session_id": "abc123"}},
)['context']

베이컨 제품 몇 가지를 보여드리겠습니다.

첫번째 제품은 2015년도에 출시된 "무첨가베이컨" 입니다. 해당 제품은 라면토핑용으로 개발된 건강지향 무첨가(아질산나트륨) 베이컨으로, 삼겹을 사용하여 만들어졌습니다. 이미 단종된 제품입니다.

두번째 제품은 2017년도에 출시된 "베이컨 SJ" 입니다. 이 제품은 무첨가베이컨 원목을 활용한 다이스 형태의 제품으로, 삼겹을 사용하여 만들어졌습니다. 이 제품 또한 단종되었습니다.

세번째 제품은 2020년도에 출시된 "비절단통베이컨" 입니다. 이 제품은 진심담은 베이컨 extension 제품으로, 삼겹을 사용하며 현재 단종되지 않고 있습니다.

네번째 제품은 2020년도에 출시된 "미추리베이컨" 입니다. 이 제품은 학교급식용으로 개발된 제품으로, 국내산 미추리 원료를 활용하여 재구성된 베이컨입니다. 현재 단종되지 않았습니다.

이 밖에도 다양한 베이컨 제품이 더 있으며, 추가 정보를 알고 싶으시면 특정 제품에 대해 검색해주세요.

| 구분  | 채널, 고객사   | 제품명                     | 단종 여부 | 런칭여부   | 사용원료           | 컨셉                                             |
|-------|---------------|---------------------------|-----------|-----------|-------------------|--------------------------------------------------|
| ODM   | 특판(농심)   | 무첨가베이컨              | 단종      | 런칭      | 삼겹              | 라면토핑용 건강지향 무첨가(아질산나트륨)베이컨 제품 개발 |
| ODM   | 다우FS       | 베이컨 SJ                  | 단종      | 런칭      | 삼겹              | 무첨가베이컨 원목을 활용한 다이스 제품    

'베이컨 제품 있어?'

In [19]:
conversational_rag_chain.invoke(
    {"input": "학교에 납품할 베이컨 제품을 생각 중인데 좋은 아이디어 없을까?"},
    config={"configurable": {"session_id": "abc123"}},
)

학교에 납품할 베이컨 제품에 대한 좋은 아이디어를 드리겠습니다. 

발렌타인데이와 같은 특별한 날에 맞춰 사용할 수 있는 제품을 고려해보세요. 제품 이름은 "아이들용 베이컨 샌드위치"로, 부드러운 식빵 사이에 "The 맛있는 베이컨"과 신선한 채소, 치즈를 넣어 만든 샌드위치입니다. 이 제품은 아이들이 좋아할 만한 맛과 영양을 고려하여 설계되었습니다. 

또한, 학교 급식에 적합한 형태로 포장될 수 있으며, 간편하게 먹을 수 있도록 구성된다면 학생들에게 좋은 반응을 얻을 수 있을 것입니다.

현재 납품할 수 있는 베이컨 제품으로는 "The 맛있는 베이컨"이 있으며, 해당 제품은 2023년도에 런칭되어 아직 단종되지 않았습니다. 전지로 만들어져 있어 품질도 우수합니다.

| 구분  | 채널, 고객사   | 제품명                     | 단종 여부 | 런칭여부   | 사용원료           | 컨셉                                             |
|-------|---------------|---------------------------|-----------|-----------|-------------------|--------------------------------------------------|
| ODM   | 온라인       | The 맛있는 베이컨        | 해당없음  | 런칭      | 전지              | 온라인 채널 판매용 전지베이컨                   |

표를 엑셀에 그대로 복사하시면 엑셀 형식으로 적용됩니다.

{'input': '학교에 납품할 베이컨 제품을 생각 중인데 좋은 아이디어 없을까?',
 'chat_history': [],
 'context': [Document(metadata={'sheet': '2023', 'source': 'The 맛있는 베이컨'}, page_content='sheet: 2023\n구분: ODM\n채널, 고객사: 온라인\n제품명: The 맛있는 베이컨\n단종 여부: 해당없음\n런칭여부: 런칭\n사용원료: 전지\n컨셉: 온라인 채널 판매용 전지베이컨')],
 'answer': '학교에 납품할 베이컨 제품에 대한 좋은 아이디어를 드리겠습니다. \n\n발렌타인데이와 같은 특별한 날에 맞춰 사용할 수 있는 제품을 고려해보세요. 제품 이름은 "아이들용 베이컨 샌드위치"로, 부드러운 식빵 사이에 "The 맛있는 베이컨"과 신선한 채소, 치즈를 넣어 만든 샌드위치입니다. 이 제품은 아이들이 좋아할 만한 맛과 영양을 고려하여 설계되었습니다. \n\n또한, 학교 급식에 적합한 형태로 포장될 수 있으며, 간편하게 먹을 수 있도록 구성된다면 학생들에게 좋은 반응을 얻을 수 있을 것입니다.\n\n현재 납품할 수 있는 베이컨 제품으로는 "The 맛있는 베이컨"이 있으며, 해당 제품은 2023년도에 런칭되어 아직 단종되지 않았습니다. 전지로 만들어져 있어 품질도 우수합니다.\n\n| 구분  | 채널, 고객사   | 제품명                     | 단종 여부 | 런칭여부   | 사용원료           | 컨셉                                             |\n|-------|---------------|---------------------------|-----------|-----------|-------------------|--------------------------------------------------|\n| ODM   | 온라인       | The 맛있는 베이컨        | 해당없음  |

In [44]:
conversational_rag_chain.invoke(
    {"input": "발렌타인데이를 맞이해서 편의점에 납품할 신메뉴 아이디어 좀 내줘"},
    config={"configurable": {"session_id": "abc123"}},
)

발렌타인데이를 맞이하여 편의점에 납품할 신메뉴 아이디어를 제안해 드리겠습니다.

첫번째 제품은 "러블리초코햄"입니다. 해당 제품은 부드러운 초콜릿 맛의 햄으로, 달콤한 초코소스를 곁들여서 사랑하는 사람들에게 특별한 간식으로 제공할 수 있습니다. 이 제품은 발렌타인데이의 달달한 분위기에 잘 어울리며, 편의점에서 간편하게 구매할 수 있도록 포장할 수 있습니다.

두번째 제품은 "로맨틱베리 소시지"입니다. 이 제품은 다양한 베리 맛이 나는 소시지로, 딸기와 블루베리의 상큼한 맛을 담았습니다. 특히, 연인과 함께 나누어 먹기 좋은 소중량 포장으로 제공될 수 있습니다.

이 외에도 다른 다양한 발렌타인데이 관련 메뉴를 고민해 볼 수 있습니다. 

| 구분  | 채널, 고객사   | 제품명                     | 단종 여부 | 런칭여부   | 사용원료           | 컨셉                                             |
|-------|---------------|---------------------------|-----------|-----------|-------------------|--------------------------------------------------|
| ODM   | 편의점       | 러블리초코햄              | 해당없음   | 런칭예정    | 후지, 지방, 초콜릿 | 발렌타인데이에 적합한 초콜릿 맛의 햄 제품       |
| ODM   | 편의점       | 로맨틱베리 소시지        | 해당없음   | 런칭예정    | 후지, 지방, 베리   | 상큼한 베리 맛이 나는 소시지 제품               |

이 표를 엑셀에 그대로 복사하시면 엑셀 형식으로 적용됩니다.

{'input': '발렌타인데이를 맞이해서 편의점에 납품할 신메뉴 아이디어 좀 내줘',
 'chat_history': [],
 'context': [Document(metadata={'sheet': '2019', 'source': '프랜들리소시지 [미니]'}, page_content='sheet: 2019\n구분: ODM\n채널, 고객사: 세븐일레븐\n제품명: 프랜들리소시지 [미니]\n단종 여부: 단종\n런칭여부: 런칭\n사용원료: 후지, 지방\n컨셉: 캐릭터 케이싱 활용 편의점 도시락 반찬용 비엔나 소시지 제품'),
  Document(metadata={'sheet': '2020', 'source': '나다운칼집비엔나'}, page_content='sheet: 2020\n구분: ODM\n채널, 고객사: 보라티알\n제품명: 나다운칼집비엔나\n단종 여부: 해당없음\n런칭여부: 뱅킹화\n사용원료: 후지, 지방\n컨셉: B2C용 소포장 나다운칼집비엔나 extension 제품'),
  Document(metadata={'sheet': '2021', 'source': '나다운델리팜[300]'}, page_content='sheet: 2021\n구분: ODM\n채널, 고객사: 선진팜\n제품명: 나다운델리팜[300]\n단종 여부: 단종\n런칭여부: 런칭\n사용원료: 후지, 지방\n컨셉: 선진팜 매장용 소포장 나다운델리팜 extension 제품'),
  Document(metadata={'sheet': '2021', 'source': 'The착한칼집비엔나[300]'}, page_content='sheet: 2021\n구분: ODM\n채널, 고객사: 선진팜\n제품명: The착한칼집비엔나[300]\n단종 여부: 단종\n런칭여부: 런칭\n사용원료: 후지, 지방\n컨셉: 선진팜 매장용 소포장 The착한칼집비엔나 extension 제품'),
  Document(metadata={'sheet': '2023', 'source': '페퍼맛소시지15(냉장)'}, page_content='s